# Notebook for error handling checks for engine, loader and strategy

In [1]:
import sys
import os
import logging
import pandas as pd
import numpy as np

# Get the Project Root Directory
project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

from src.core.loader import load_data, Error_handling
from src.core.engine import run
from src.strategy.strategy import Strategy

# Configure Logging to display in Notebook
logging.basicConfig(
    stream=sys.stdout, 
    level=logging.INFO, 
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    force=True
)

logger = logging.getLogger("Notebook_Test")
logger.info("Logging configured. Ready for testing.")

2025-11-26 12:26:23,695 - Notebook_Test - INFO - Logging configured. Ready for testing.


In [2]:
# --- Test 1.1: Missing File ---
print(">>> TEST 1: Missing File Handling")
try:
    load_data("ghost_file.csv")
except FileNotFoundError as e:
    print(f"SUCCESS: Caught missing file error: {e}")
except Exception as e:
    print(f"FAILED: Wrong error type: {type(e)}")

# --- Test 1.2: Bad Schema (Validation) ---
print("\n>>> TEST 2: Schema Validation")
# Create a bad csv
bad_csv = pd.DataFrame({'price': [1, 2, 3], 'vol': [10, 10, 10]}) # Missing 'date', 'open', etc.
bad_csv.to_csv('bad_schema.csv', index=False)

try:
    Error_handling('bad_schema.csv')
except ValueError as e:
    print(f"SUCCESS: Caught bad schema error: {e}")
finally:
    if os.path.exists('bad_schema.csv'): os.remove('bad_schema.csv')

# --- Test 1.3: Success Case ---
print("\n>>> TEST 3: Successful Load")
# Create a good csv
dates = pd.date_range(start='2023-01-01', periods=100, freq='D')
good_csv = pd.DataFrame({
    'date': dates,
    'open': np.random.rand(100) * 100,
    'high': np.random.rand(100) * 105,
    'low': np.random.rand(100) * 95,
    'close': np.random.rand(100) * 100,
    'volume': np.random.randint(100, 1000, 100)
})
good_csv.to_csv('good_data.csv', index=False)

df = load_data('good_data.csv')
if os.path.exists('good_data.csv'): 
    print(f"SUCCESS: Loaded good data")
    os.remove('good_data.csv')

>>> TEST 1: Missing File Handling
2025-11-26 12:26:25,455 - src.core.loader - ERROR - File not found: ghost_file.csv
SUCCESS: Caught missing file error: File not found: ghost_file.csv

>>> TEST 2: Schema Validation
2025-11-26 12:26:25,470 - src.core.loader - ERROR - Missing columns: ['date', 'open', 'high', 'low', 'close', 'volume']
SUCCESS: Caught bad schema error: Missing columns: ['date', 'open', 'high', 'low', 'close', 'volume']

>>> TEST 3: Successful Load
2025-11-26 12:26:25,514 - src.core.loader - INFO - Successfully loaded 100 rows
SUCCESS: Loaded good data


In [3]:
# --- Configuration Setup for Tests ---
mock_cfg = {
    'indicators': {
        'rsi_period': 14,
        'short_composite_rsi': 10,
        'long_composite_rsi': 20,
        'hurst_window': 20
    },
    'entry_thresholds': {
        'rsi_low': 30,
        'rsi_high': 70,
        'hurst_threshold': 0.5
    },
    'exits': {
        'max_bars_in_trade': 10,
        'max_profitable_closes': 5,
        'composite_rsi_threshold': 80
    }
}

In [5]:
print(">>> TEST 4: Strategy Initialization Safety Checks")

try:
    # Initialize Strategy with Config
    strategy = Strategy(mock_cfg)

    # Test  Missing Columns
    print("\n--- Testing Missing Columns ---")
    
    # Create a dataframe MISSING the 'rsi' column
    incomplete_df = pd.DataFrame({'open': [100], 'close': [105], 'hurst': [0.6]})
    state = {'open_position': False}
    
    # This should NOT crash, but it should log an error and return False
    result = strategy.entry_signal(incomplete_df, 0, state)
    
    print(f"Result with missing RSI: {result}")

except ImportError as e:
    print(f"SKIPPED: Could not import sub-modules (signals/exits). Error: {e}")
except Exception as e:
    print(f"FAILED: Strategy Logic Error: {e}")

>>> TEST 4: Strategy Initialization Safety Checks

--- Testing Missing Columns ---
2025-11-26 12:27:01,066 - src.strategy.strategy - ERROR - RSI is NOT in the Dataframe!
Result with missing RSI: False


In [6]:
print(">>> TEST 5: Engine Runtime Robustness")

# Generate Data
dates = pd.date_range(start='2024-01-01', periods=100, freq='D')
df = pd.DataFrame({
    'date': dates,
    'open': np.full(100, 100.0), 
    'high': np.full(100, 105.0),
    'low': np.full(100, 95.0),
    'close': np.full(100, 100.0),
    'volume': np.random.uniform(1000, 5000, size=100)
})

# Initialize Strategy
strategy = Strategy(mock_cfg)

# SABOTAGE THE DATA
# Set 'close' to NaN, so df['rsi'] will be NaN at these indices
df.loc[40:45, 'close'] = np.nan 

# Run Engine
try:
    results = run(df, strategy)
        
except Exception as e:
    print(f"CRITICAL FAILURE: Engine crashed: {e}")

>>> TEST 5: Engine Runtime Robustness
2025-11-26 12:27:05,418 - src.core.engine - INFO - Starting backtest on 100 rows.
2025-11-26 12:27:05,421 - src.core.engine - WARNING - Indicator failure: 'Close'
